In [1]:
# 📦 Standard imports
import logging
import time
import re
from pathlib import Path
import pandas as pd
import json

# 📄 Docling imports for PDF conversion, OCR, tables, etc.
from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import (
    PdfPipelineOptions,
    TesseractCliOcrOptions
)
from docling.document_converter import DocumentConverter, PdfFormatOption
from docling_core.types.doc import ImageRefMode, TableItem

# ✂️ Chunking and HuggingFace tokenizer
from docling_core.transforms.chunker.tokenizer.huggingface import HuggingFaceTokenizer
from transformers import AutoTokenizer, AutoModel
import torch

# 📊 Vector handling using Qdrant
import numpy as np
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct

from docling.chunking import HybridChunker

# 🔖 Markdown serializer from Docling Core
from docling_core.transforms.serializer.markdown import MarkdownDocSerializer

# ✅ Helper: filter lines in Markdown that are too short or meaningless
def is_valid_line(line: str) -> bool:
    line = line.strip()
    if not line:
        return False
    if len(line) < 5:
        return False
    if re.fullmatch(r"[|\\/\-=_*~.\s]+", line):
        return False
    if sum(c.isalnum() for c in line) / max(len(line), 1) < 0.3:
        return False
    return True

# ✅ Clean up Markdown before chunking
def clean_markdown(md: str) -> str:
    lines = md.splitlines()
    filtered_lines = [line for line in lines if is_valid_line(line)]
    return "\n".join(filtered_lines)

# 🔢 Convert texts into normalized dense embeddings using a HuggingFace model
def embed(texts, model, tokenizer, batch_size=8):
    embeddings = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i + batch_size]
        inputs = tokenizer(batch, padding=True, truncation=True, return_tensors="pt")
        with torch.no_grad():
            model_output = model(**inputs)
        emb = model_output.last_hidden_state[:, 0]  # CLS token
        embeddings.extend(emb.cpu().numpy())
    embeddings = np.array(embeddings).astype("float32")
    embeddings /= np.linalg.norm(embeddings, axis=1, keepdims=True)
    return embeddings

# 🚀 Main chunking, embedding, and saving logic
def save_chunks(doc, output_dir: Path, doc_filename: str):
    EMBED_MODEL_ID = "BAAI/bge-m3"
    MAX_TOKENS = 8192

    tokenizer = HuggingFaceTokenizer(
        tokenizer=AutoTokenizer.from_pretrained(EMBED_MODEL_ID),
        max_tokens=MAX_TOKENS,
    )
    chunker = HybridChunker(tokenizer=tokenizer, merge_peers=True)
    chunk_iter = chunker.chunk(dl_doc=doc)

    all_chunks = []
    for i, chunk in enumerate(chunk_iter, 1):
        enriched_text = chunker.contextualize(chunk=chunk).strip()
        if len(enriched_text) < 100:
            continue
        all_chunks.append({
            "index": i,
            "text": enriched_text,
            "chunk_id": f"{doc_filename}_chunk_{i:03}"
        })

    (output_dir / "chunks").mkdir(exist_ok=True)
    metadata_path = output_dir / f"{doc_filename}_metadata.json"
    with open(metadata_path, "w", encoding="utf-8") as f:
        json.dump(all_chunks, f, ensure_ascii=False, indent=2)

    embed_tokenizer = AutoTokenizer.from_pretrained(EMBED_MODEL_ID)
    embed_model = AutoModel.from_pretrained(EMBED_MODEL_ID)
    texts = [chunk["text"] for chunk in all_chunks]
    vectors = embed(texts, embed_model, embed_tokenizer)

    qdrant = QdrantClient(path=str(output_dir / f"{doc_filename}_qdrant"))
    collection_name = f"{doc_filename}_collection"
    qdrant.recreate_collection(
        collection_name=collection_name,
        vectors_config=VectorParams(size=vectors.shape[1], distance=Distance.COSINE)
    )

    qdrant.upload_points(
        collection_name=collection_name,
        points=[
            PointStruct(id=i, vector=vec.tolist(), payload={"text": chunk["text"]})
            for i, (vec, chunk) in enumerate(zip(vectors, all_chunks))
        ]
    )

    logging.info(f"Saved {len(all_chunks)} chunks and vectors to Qdrant at {output_dir}")

# 📜 Main document extraction logic from PDF to Markdown & images
def extract_from_pdf(input_pdf_path: Path, output_dir: Path):
    is_arabic = "AR" in input_pdf_path.name.upper()
    logging.basicConfig(level=logging.INFO)
    _log = logging.getLogger(__name__)

    pipeline_options = PdfPipelineOptions()
    pipeline_options.do_ocr = True
    pipeline_options.ocr_options = TesseractCliOcrOptions(
        force_full_page_ocr=True,
        lang=["ara"] if is_arabic else ["eng"]
    )
    pipeline_options.do_table_structure = True
    pipeline_options.table_structure_options.do_cell_matching = True
    pipeline_options.images_scale = 3.0
    pipeline_options.generate_page_images = True
    pipeline_options.generate_picture_images = True

    doc_converter = DocumentConverter(
        format_options={
            InputFormat.PDF: PdfFormatOption(pipeline_options=pipeline_options)
        }
    )

    start_time = time.time()
    conv_res = doc_converter.convert(input_pdf_path)
    doc = conv_res.document
    output_dir.mkdir(parents=True, exist_ok=True)
    doc_filename = conv_res.input.file.stem

    image_dir = output_dir / "images"
    image_dir.mkdir(exist_ok=True)

    direction_tag = "dir='rtl'" if "AR" in doc_filename.upper() else ""

    # ⚡️ Apply MarkdownDocSerializer correctly
    serializer = MarkdownDocSerializer(doc=doc)
    ser_result = serializer.serialize()
    raw_md = ser_result.text

    cleaned_md = clean_markdown(raw_md)
    md_parts = [f"# Extracted Content from {doc_filename}\n", f"<div {direction_tag}>\n", cleaned_md, "</div>"]

    for i, table in enumerate(doc.tables):
        df = table.export_to_dataframe()
        df.to_csv(output_dir / f"{doc_filename}-table-{i+1}.csv", index=False)
        table_md = df.to_markdown(index=False)
        md_parts.append(f"\n\n## Table {i+1}\n")
        md_parts.append(table_md)
        img_name = f"{doc_filename}-table-img-{i+1}.png"
        img_path = image_dir / img_name
        table.get_image(doc).save(img_path, "PNG")

    for i, page in enumerate(doc.pages):
        if hasattr(page, "image") and page.image:
            page_img_name = f"{doc_filename.lower()}_page_{i+1}_img.png"
            page_img_path = image_dir / page_img_name
            page.image.save(page_img_path, "PNG")
        else:
            _log.warning(f"Skipping page {i+1} — no image found.")

    for i, picture in enumerate(getattr(doc, "pictures", [])):
        try:
            picture_name = f"{doc_filename.lower()}_picture_{i+1}.png"
            picture_path = image_dir / picture_name
            picture.get_image(doc).save(picture_path, "PNG")
        except Exception as e:
            _log.warning(f"Failed to save embedded image {i+1}: {e}")

    final_md = "\n".join(md_parts)
    (output_dir / f"{doc_filename}.md").write_text(final_md, encoding="utf-8")

    save_chunks(doc, output_dir, doc_filename)

    elapsed = time.time() - start_time
    _log.info(f"Extraction completed in {elapsed:.2f} seconds")

if __name__ == "__main__":
    inputs = [
        ("PIF Annual Report 2021-ar.pdf", "PIF Annual Report 2021.pdf", "2021"),
        ("PIF Annual Report 2022-ar.pdf", "PIF Annual Report 2022.pdf", "2022"),
        ("PIF-2023-Annual-Report-AR.pdf", "PIF-2023-Annual-Report-EN.pdf", "2023")
    ]

    for ar_file, en_file, year in inputs:
        output_ar = Path(f"output_ar_{year}")
        output_en = Path(f"output_en_{year}")
        extract_from_pdf(Path(ar_file), output_ar)
        extract_from_pdf(Path(en_file), output_en)
any


c:\Users\Mohamed\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:docling.datamodel.document:detected formats: [<InputFormat.PDF: 'pdf'>]
INFO:docling.document_converter:Going to convert document batch...
INFO:docling.document_converter:Initializing pipeline for StandardPdfPipeline with options hash 0e668d04436ed0287f29833c18512eb0
INFO:docling.models.factories.base_factory:Loading plugin 'docling_defaults'
INFO:docling.models.factories:Registered ocr engines: ['easyocr', 'ocrmac', 'rapidocr', 'tesserocr', 'tesseract']
INFO:docling.models.tesseract_ocr_cli_model:command: tesseract --list-langs
INFO:docling.utils.accelerator_utils:Accelerator device: 'cpu'
INFO:docling.utils.accelerator_utils:Accelerator device: 'cpu'
INFO:docling.models.factories.base_factory:Loading plu

<function any(iterable, /)>

In [6]:
from qdrant_client import QdrantClient
from transformers import AutoTokenizer, AutoModel
import numpy as np
import torch
from pathlib import Path

def embed_query(text, model, tokenizer):
    tokens = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        model_output = model(**tokens)
    vec = model_output.last_hidden_state[:, 0].cpu().numpy()
    vec = vec / np.linalg.norm(vec)
    return vec.astype("float32")

# ✅ Set up paths and model
output_dir_ar = Path("output_ar_2023")
doc_filename_ar = "PIF-2023-Annual-Report-AR"
collection_name_ar = f"{doc_filename_ar}_collection"

qdrant = QdrantClient(path=str(output_dir_ar / f"{doc_filename_ar}_qdrant"))

# Load embedding model
model_id = "BAAI/bge-m3"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModel.from_pretrained(model_id)

# ✅ Arabic query (replace with your question)
arabic_query = "ما هي رؤية صندوق الاستثمارات العامة؟"
query_vector = embed_query(arabic_query, model, tokenizer)

# 🔍 Search Qdrant
results = qdrant.search(
    collection_name=collection_name_ar,
    query_vector=query_vector[0],
    limit=5
)

# ✅ Display results
for result in results:
    print(f"\n🔹 Score: {result.score:.3f}")
    print(result.payload["text"])



🔹 Score: 0.721
الرؤية
يسعى صندوق الاستثمارات العامة إلى أن يكون قوة محركة للاستثمار. وأن يصبح الجهة الاستثمارية الأكثر تآثيرا فى العالم. وأن يدعم إطلاق قطاعات وفرص جديدة تساهم في رسم ملامجح مستقبل الاقتصاد العالمى. ويدفع بذلك عجلة التحول الاقتصادى فى المملكة العربية السعودية. 0

🔹 Score: 0.655
التركيز والأهداف
مع التقدّم الذي أحرزه صندوق الاستثمارات العامة خلال السنوات الثلاث الأولى. في تنفيذ استراتيجية صندوق الاستراتيجى لتنمية وتنويع اقتصاد المملكة العربية السعودية. والاستمرار فى الاستثمار محليًا لاستكشاف إمكانات النمو فى القطاعات الوطنية ذات الأولوية. والاستثمار دوليًا لتوسيع وتنويع قاعدة أصوله وتحقيق عوائد مستدامة تعزز الاقتصاد المحلى.
حدد صندوق الاستثمارات العامة أهدافاً طموحة ضمن استراتيجيتة تشمل؛ زيادة إجمالى الأصول المدارة لتصل إلى حوالي 4 تريليون ريال سعودي. ورفع مساهمة الصندوق وشركات محفظته في الناتج المحلي الإجمالي غير النفطي إلى 2 تريليون ريال سعودي تراكمي. بالإضافة إلى ذلك. يستهدف صندوق الاستثمارات العامة زيادة مساهمته وشركات محفظته فى المحتوى المحلى إلى 9660. وزيادة الاست

C:\Users\Mohamed\AppData\Local\Temp\ipykernel_8616\3104064960.py:32: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = qdrant.search(


In [3]:
from qdrant_client import QdrantClient
from transformers import AutoTokenizer, AutoModel
import numpy as np
import torch
from pathlib import Path

def embed_query(text, model, tokenizer):
    tokens = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        model_output = model(**tokens)
    vec = model_output.last_hidden_state[:, 0].cpu().numpy()
    vec = vec / np.linalg.norm(vec)
    return vec.astype("float32")

# ✅ Load model once
model_id = "BAAI/bge-m3"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModel.from_pretrained(model_id)

# ✅ Arabic query
arabic_query = "ما هي رؤية صندوق الاستثمارات العامة؟"
query_vector = embed_query(arabic_query, model, tokenizer)

# ✅ Report file names (same ones you used during chunking)
year_to_filename = {
    "2021": "PIF Annual Report 2021-ar",
    "2022": "PIF Annual Report 2022-ar",
    "2023": "PIF-2023-Annual-Report-AR"
}

# ✅ Search across all collections
for year, doc_filename in year_to_filename.items():
    output_dir = Path(f"output_ar_{year}")
    collection_name = f"{doc_filename}_collection"
    qdrant_path = output_dir / f"{doc_filename}_qdrant"

    try:
        qdrant = QdrantClient(path=str(qdrant_path))
        results = qdrant.search(
            collection_name=collection_name,
            query_vector=query_vector[0],
            limit=1,
            with_payload=True
        )
        print(f"\n📘 Results from {collection_name}:")
        for result in results:
            print(f"\n🔹 Score: {result.score:.3f}")
            print(result.payload.get("text", "[No text]"))

    except Exception as e:
        print(f"⚠️ Failed to search {collection_name}: {e}")



📘 Results from PIF Annual Report 2021-ar_collection:

🔹 Score: 0.680
الرؤية أن يكون الصندوق
للاستثمار والجبة الاستثمارية الأكثر تأثيراً على مستوى العالم»
وأن يدفع عجلة التحول الاقتصادي في المملكة العربية السعودية. وأن يدعم إطلاق قطاعات وفرص جديدة تساعد على رسم ملامح مستقبل الاقتصاد
صندوق الاستثمارات العامة هو الذراع الاستثماري
فإنه في ذات الوقت يسعى لإحداث تأثير إيجابي ومستدام من خلال استراتيجيته المتينة الهادفة إلى توسيع محفظته من الأصول المحلية والعالمية» والاستثمار في القطاعات والأسواق الاستراتيجية. وتوطين التقنيات والخبرات الحديثة. وإقامة شراكات استراتيجية, وإطلاق المبادرات لبناء مستقبل أكثر إشراقاً للوطن وللعالم.
82
00

📘 Results from PIF Annual Report 2022-ar_collection:

🔹 Score: 0.698
برنامج صندوق الاستثمارات العامة لتحقيق الرؤية
يؤدي صندوق الاستثمارات العامة دوراً استراتيجياً حيوياً ضمن رؤبة المملكة 2030 الهادفة لإحداث تحول اقتصادي وتغيير إيجابي مستدام في المملكة.
يدعم صندوق الاستثمارات العامة تنمية الاقتصاد الدولية. كما هيدف إلى تعظيم العوائد المستدامة لاقتصاد المملكة.
تحقيق

C:\Users\Mohamed\AppData\Local\Temp\ipykernel_8616\882296862.py:39: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = qdrant.search(


In [19]:
from qdrant_client import QdrantClient
from transformers import AutoTokenizer, AutoModel
import numpy as np
import torch
from pathlib import Path

def stream_text(text, delay=0.05):
    for word in text.split():
        print(word, end=" ", flush=True)
        time.sleep(delay)
    print()

# ✅ Embed query function
def embed_query(text, model, tokenizer):
    tokens = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        model_output = model(**tokens)
    vec = model_output.last_hidden_state[:, 0].cpu().numpy()
    vec = vec / np.linalg.norm(vec)
    return vec.astype("float32")

# ✅ Load model and tokenizer
model_id = "BAAI/bge-m3"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModel.from_pretrained(model_id)

# ✅ Arabic query
arabic_query = "نيوم؟"
query_vector = embed_query(arabic_query, model, tokenizer)

# ✅ Define PDF collections
year_to_filename = {
    "2021": "PIF Annual Report 2021-ar",
    "2022": "PIF Annual Report 2022-ar",
    "2023": "PIF-2023-Annual-Report-AR"
}

# ✅ Gather all results
all_results = []

for year, doc_filename in year_to_filename.items():
    output_dir = Path(f"output_ar_{year}")
    collection_name = f"{doc_filename}_collection"
    qdrant_path = output_dir / f"{doc_filename}_qdrant"

    try:
        qdrant = QdrantClient(path=str(qdrant_path))
        results = qdrant.search(  # You can also use `query_points` in the future
            collection_name=collection_name,
            query_vector=query_vector[0],
            #limit=2,
            with_payload=True
        )
        for r in results:
            all_results.append((r, collection_name))  # ⬅️ Save result with its source

    except Exception as e:
        print(f"⚠️ Failed to search {collection_name}: {e}")

# ✅ Print best result across all
if all_results:
    best_result, source_collection = max(all_results, key=lambda x: x[0].score)
    print("\n🏆 Best overall result:", flush=True)
    print(f"🗂️ From: {source_collection}", flush=True)
    print(f"🔹 Score: {best_result.score:.3f}", flush=True)
    print("📝 Answer:", flush=True)
    stream_text(best_result.payload.get("text", "[No text]"))



🏆 Best overall result:
🗂️ From: PIF Annual Report 2021-ar_collection
🔹 Score: 0.610
📝 Answer:
انطلاقة التغيير يقع 

C:\Users\Mohamed\AppData\Local\Temp\ipykernel_8616\3731838468.py:48: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = qdrant.search(  # You can also use `query_points` in the future


مشروع نيوم الذي يعني (المستقبل الجديد) بالقرب من البحر الأحمر في شمال غرب المملكة العربية السعودية» ويهدف إلى تحفيز ريادة الأعمال والابتكارات السابقة لعصرهاء حيث يمثل المشروع تجسيداً للتقدم البشري ورؤية لما قد يبدو عليه المستقبل الجديد. سيركز مشروع نيوم على قطاعات الطاقة المتجددة و الصحة والمياه - ٠الاستفادة‏ من انخفاض تكاليف توليد الطاقة النظيفة والمتجددة. - ٠‏ تعزيز الابتكارات الطبية لزيادة متوسط العمر المتوقع. - ٠‏ إجاد قوة إقليمية في إنتاج المياه وتخزينها ومركز للتميز في تكنولوجيا المياه العالمية. تبلغ مساحة مشروع نيوم حوالي 26,500 كم" . وتتمتع بالمناخ الأكثر اعتدالًا في المنطقة والمناظر الطبيعية الخلابة. من الصحاري ذات الرمال الحمراء إلى الجبال الشامخة. وخط ساحلي يمتد على البحر الأحمر إلى خليج العقبة بأكثر من 450 كمء وتُعتبر مياهها موطنًا لبعض الشعب المرجانية الأكثر تكيفاً في العالم . مما يجعلها كنرًا دفينًا للحياة البحرية. بالإضافة إلى العديد من المزايا التي تجعل نيوم مكاناً مميزاً. سيتم تشغيل نيوم من خلال الطاقة المتجددة فقطء وربطها بنظام بيئي للتنقل الممستدام وتحيط بها الطبيعة 

In [18]:
from qdrant_client import QdrantClient
from transformers import AutoTokenizer, AutoModel
import numpy as np
import torch
from pathlib import Path

# ✅ Typing effect function
def stream_text(text, delay=0.05):
    for word in text.split():
        print(word, end=" ", flush=True)
        time.sleep(delay)
    print()


# ✅ Embed query
def embed_query(text, model, tokenizer):
    tokens = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        model_output = model(**tokens)
    vec = model_output.last_hidden_state[:, 0].cpu().numpy()
    vec = vec / np.linalg.norm(vec)
    return vec.astype("float32")

# ✅ Load model and tokenizer
model_id = "BAAI/bge-m3"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModel.from_pretrained(model_id)

# ✅ English query
query = "What is the vision of the Public Investment Fund?"
query_vector = embed_query(query, model, tokenizer)

# ✅ Collection mapping
year_to_filename = {
    "2021": "PIF Annual Report 2021",
    "2022": "PIF Annual Report 2022",
    "2023": "PIF-2023-Annual-Report-EN"
}

# ✅ Store best result
best_result = None
best_score = -1
best_collection = None

for year, doc_filename in year_to_filename.items():
    output_dir = Path(f"output_en_{year}")
    qdrant_path = output_dir / f"{doc_filename}_qdrant"
    collection_name = f"{doc_filename}_collection"

    try:
        qdrant = QdrantClient(path=str(qdrant_path))
        results = qdrant.search(
            collection_name=collection_name,
            query_vector=query_vector[0],
            #limit=1,
            with_payload=True
        )
        if results:
            result = results[0]
            if result.score > best_score:
                best_result = result
                best_score = result.score
                best_collection = collection_name
    except Exception as e:
        print(f"⚠️ Failed to search {collection_name}: {e}")

# ✅ Final output
if best_result:
    print("\n🏆 Best overall English result:",flush=True)
    print(f"🗂️ From: {best_collection}",flush=True)
    print(f"🔹 Score: {best_score:.3f}",flush=True)
    print(best_result.payload["text"],flush=True)
else:
    print("⚠️ No results found in any collection.")



🏆 Best overall English result:
🗂️ From: PIF-2023-Annual-Report-EN_collection
🔹 Score: 0.686
PIF VISION REALIZATION PROGRAM
The Public Investment Fund is a cornerstone in Saudi Vision 2030, tasked with driving sustainable and transformative economic change. Focused on strengthening the local economy and building its international asset portfolio, PIF is dedicated to maximizing sustainable returns and fostering economic diversification.


C:\Users\Mohamed\AppData\Local\Temp\ipykernel_8616\1867601680.py:52: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = qdrant.search(


In [ ]:
from qdrant_client import QdrantClient
from transformers import AutoTokenizer, AutoModel
import numpy as np
import torch
from pathlib import Path
import time
import re
import warnings




# ✅ Typing effect function (word-by-word)
def stream_text(text, delay=0.05):
    for word in text.split():
        print(word, end=" ", flush=True)
        time.sleep(delay)
    print()

# ✅ Clean answer text
def clean_answer(text):
    # Remove temporary file paths and unwanted characters
    text = re.sub(r'C:\\Users\\.*?\.py:\d+:.*?\n', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# ✅ Embed query function
def embed_query(text, model, tokenizer):
    tokens = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        model_output = model(**tokens)
    vec = model_output.last_hidden_state[:, 0].cpu().numpy()
    vec = vec / np.linalg.norm(vec)
    return vec.astype("float32")

# ✅ Load model and tokenizer
model_id = "BAAI/bge-m3"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModel.from_pretrained(model_id)

# ✅ Arabic query
arabic_query = "نيوم2023؟"
query_vector = embed_query(arabic_query, model, tokenizer)

# ✅ Define PDF collections
year_to_filename = {
    "2021": "PIF Annual Report 2021-ar",
    "2022": "PIF Annual Report 2022-ar",
    "2023": "PIF-2023-Annual-Report-AR"
}

# ✅ Gather all results
all_results = []

for year, doc_filename in year_to_filename.items():
    output_dir = Path(f"output_ar_{year}")
    collection_name = f"{doc_filename}_collection"
    qdrant_path = output_dir / f"{doc_filename}_qdrant"

    try:
        qdrant = QdrantClient(path=str(qdrant_path))
        results = qdrant.search(  # You can also use `query_points` in the future
            collection_name=collection_name,
            query_vector=query_vector[0],
            limit=5,
            with_payload=True
        )
        for r in results:
            all_results.append((r, collection_name))  # ⬅️ Save result with its source

    except Exception as e:
        print(f"⚠️ Failed to search {collection_name}: {e}", flush=True)

# ✅ Print best result across all
if all_results:
    best_result, source_collection = max(all_results, key=lambda x: x[0].score)
    answer_text = clean_answer(best_result.payload.get("text", "[No text]"))

    print("\n" + "═" * 50)
    print("🏆 Best Answer Found".center(50))
    print("═" * 50)
    print(f"📘 Source     : {source_collection}")
    print(f"📊 Score      : {best_result.score:.3f}")
    print("📝 Answer     :\n")
    stream_text(answer_text, delay=0.03)  # Typing effect
    print("═" * 50)
else:
    print("⚠️ No results found in any collection.", flush=True)
warnings.filterwarnings("ignore", category=DeprecationWarning)



══════════════════════════════════════════════════
               🏆 Best Answer Found                
══════════════════════════════════════════════════
📘 Source     : PIF-2023-Annual-Report-AR_collection
📊 Score      : 0.515
📝 Answer     :

الرؤية الطموحة التي ستشكل المستقبل الجديد تُشرع نيوم؛ المركز العالمي المبتكر الذي يُقام على مساحة شاسعة تبلغ 26,500 كيلومةر مربع في شمال غرب السعودية بمنطقة تبوك؛ في تحويل رؤيتها الطموحة إلى واقع حي. يضم هذا المشروع مدينتين متميزتين: "ذا لاين". مدينة خطية مبتكرة تعمل بالكامل على الطاقة المتجددة, و"أوكساجون" مدينة متطورة تركز على الخدمات اللوجستية والصناعات المتقدمة والنظيفة. إلى جانب ذلك. توفر نيوم وجهات سياحية وبقيادة فريق من الخبراء العالميين, تعمل نيوم. وهي مملوكة بالكامل لصندوق الاستثمارات العامة على تطوير المنطقة عبر 14 قطاعا اقتصاديًا رئيسيًا. ويدعم صندوق نيوم للاستثمار. الذراع الاستثماري الاستراتيجي لنيوم, تحقيق هذه الرؤية من خلال ضخ الاستثمارات اللازمة في المشروع. يعتبر الابتكار ركيزة أساسية في نيوم. فمن خلال شركتها الفرعية "تونومس" أعلنت 